In [1]:
import time
from datetime import datetime
from tqdm.keras import TqdmCallback
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import shutil
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm_notebook as tqdm

import matplotlib.pyplot as plt

from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.models import RNNModel, TCNModel, TransformerModel, NBEATSModel, BlockRNNModel
from darts.utils.missing_values import fill_missing_values
from darts.utils.likelihood_models import GaussianLikelihoodModel

from darts.metrics import mape, mse
from darts.utils.statistics import check_seasonality, plot_acf
from darts.datasets import AirPassengersDataset, SunspotsDataset
from darts.utils.timeseries_generation import datetime_attribute_timeseries

from math import sqrt
from time import time

from datetime import timedelta

import warnings
warnings.filterwarnings("ignore")
import logging
logging.disable(logging.CRITICAL)

%pylab inline

Importing plotly failed. Interactive plots will not work.


Populating the interactive namespace from numpy and matplotlib


# The notebook leverages coinbase daily data
- open, low, high, close, volume
- unlike the other notebook where we have to create these daily candles

### Read in data

In [2]:
bitcoin_df = pd.read_csv("../data/historic_crypto_prices - bitcoin_jan_2017_sep_4_20201.csv", 
    index_col=0,
    parse_dates=True)

In [3]:
def convert_currency_to_float(input_df):
    for col in input_df.columns:
        input_df[col] = input_df[col].apply(lambda x: x.replace('$','').replace(",","")).astype(float)
    return input_df

In [4]:
bitcoin_df = convert_currency_to_float(bitcoin_df)

In [5]:
bitcoin_df

,open,high,low,close,volume,market_cap
date,,,,,,
2021-09-04,50009.33,50545.58,49548.78,49944.63,3.747133e+10,9.392861e+11
2021-09-03,49288.25,50982.27,48386.09,50025.37,4.320618e+10,9.407572e+11
2021-09-02,48807.85,50343.42,48652.32,49327.72,3.950807e+10,9.275909e+11
2021-09-01,47099.77,49111.09,46562.44,48847.03,3.913940e+10,9.185015e+11
2021-08-31,47024.34,48189.55,46750.09,47166.69,3.473036e+10,8.868699e+11
...,...,...,...,...,...,...
2017-01-05,1156.73,1191.10,910.42,1013.38,5.101990e+08,1.630025e+10
2017-01-04,1044.40,1159.42,1044.40,1154.73,3.449460e+08,1.857187e+10
2017-01-03,1021.60,1044.08,1021.60,1043.84,1.851680e+08,1.678637e+10


In [6]:
# [ time, low, high, open, close, volume ]
# 9/4
# 49400, 50558.75, 50025, 49942.98, 7595.48816499],

In [7]:
bitcoin_df = bitcoin_df.sort_index()
# don't get market cap from coinbase api
bitcoin_df.drop('market_cap' ,inplace=True, axis=1)

In [8]:
etherum_df = pd.read_csv("../data/historic_crypto_prices - etherum_jan_2017_sept_4_2021.csv", 
    index_col=0,
    parse_dates=True)

In [9]:
etherum_df.tail()

,open,high,low,close,volume,market_cap
date,,,,,,
2017-01-05,$11.29,$11.89,$9.40,$10.25,"$41,557,400","$898,497,892"
2017-01-04,$9.71,$11.28,$9.56,$11.25,"$41,051,200","$985,515,893"
2017-01-03,$8.37,$10.00,$8.32,$9.73,"$33,625,200","$851,512,055"
2017-01-02,$8.17,$8.44,$8.05,$8.38,"$14,579,600","$733,331,654"
2017-01-01,$7.98,$8.47,$7.98,$8.17,"$14,731,700","$715,049,208"


In [10]:
etherum_df = convert_currency_to_float(etherum_df)

In [11]:
# don't get market cap from coinbase api
etherum_df = etherum_df.sort_index()
etherum_df.drop('market_cap' ,inplace=True, axis=1)

### Create the simulator

In [30]:
class BollingerBandsSimulator:

    def __init__(self, df_original, from_date, period, 
                 window, no_of_std, ml_lookback_windows,  ml_prediction_n_days, model_name="TCN", additional_dfs=[],
                 stop_loss_pct=.10, figsize=None):
        """
        df_original: The input dataframe containing candles we want to build bollinger bands from and predict.
            Assumes thes date is the index
        from_date: The start date to slice the df_original by
        period: time frequency to build candles if needed and transform timeseries dataset
        window: lookback window for bollinger bands + roling mean
        no_of_std: number of std for bollinger bands
        ml_lookback_windows: one lookback window per ML model
        ml_prediction_n_days: n days in the future to predcit
        additional_dfs: Additional DFs. Assumed to be the same dates are the df_original and have a 'close' col
        stop_loss_pct: the percent under/over our short/buy to keep a stop at
        """
        self.df = df_original
        self.from_date = from_date
        self.period = period
        self.window = window
        self.no_of_std = no_of_std
        self.figsize = figsize
        self.ml_lookback_windows = ml_lookback_windows
        self.max_looback = max(ml_lookback_windows)
        
        self.ml_models_dict = {}
        self.have_trained_ml_models = False
        self.buy_entry_price = None
        self.short_entry_price = None
        self.ml_prediction_n_days = ml_prediction_n_days # the number of days in the future to predict
        self.additional_dfs = additional_dfs
        self.ml_train_cols = ['open', 'high', 'low', 'Rolling Mean', 'volume']
        self.model_name = model_name # what type of ML model to train
        self.pred_col = 'close'
        self.stop_loss_price = 0 # Price at which we get out of our position
        self.stop_loss_pct = .10 # percent to trail our buy/short until we get out
        self.first_run = True # if first run, train the models longer
        
        self.start_time = time.time()

        
        
    def _scale_time_series_df_and_time_cols(self, input_df,  time_cols = ['year', 'month', 'day']):
        ts_transformers = {}
        ts_stacked_series = None
        ts_transformers, ts_stacked_series = self._scale_time_series_df(input_df)
            
        
        # build year and month and day series:
        for col in time_cols:
            transformer = Scaler()
            transformed_series = transformer.fit_transform(datetime_attribute_timeseries(ts_stacked_series, attribute=col))
            ts_transformers[col] = transformer    

            ts_stacked_series = ts_stacked_series.stack(transformed_series)
             

        return ts_transformers, ts_stacked_series, TimeSeries.from_series(input_df[self.pred_col],  freq=self.period)
    
    def _scale_time_series_df(self, input_df, use_pred_col=False):
        """
        Scale an input time series col from 0 to 1
        
        input_df: the DF that contains the col
        use_pred_col: if we are transforming additional DFs, we can use the pred col 'close' for them
        """
        ts_transformers = {}
        ts_stacked_series = None
        cols_to_transform = self.ml_train_cols.copy()
        # if we have additional DFs, we can include their close price
        if use_pred_col:
            cols_to_transform.append(self.pred_col)
        for col in cols_to_transform:
            transformer = Scaler()

            transformed_series = transformer.fit_transform(fill_missing_values(TimeSeries.from_series(input_df[col],
                                                                                                      freq=self.period)))
            ts_transformers[col] = transformer    

            if ts_stacked_series:
                ts_stacked_series = ts_stacked_series.stack(transformed_series)

            else:
                ts_stacked_series = transformed_series
        return ts_transformers, ts_stacked_series
    
    def _add_additional_training_dfs(self, ts_stacked_series, additional_dfs, verbose = False):
        """
        Scale any additional DFs provided (such as ETHER)
        
        ts_stacked_series: the current scaled lists from the df_original provided
        additional_dfs: additional dataframes that have been sliced for the correct date
        """
        all_ts_stacked_series = None
        for df in additional_dfs:
            additional_ts_transformers, additional_ts_stacked_series = self._scale_time_series_df(df, use_pred_col=True)
            if all_ts_stacked_series is None:
                if verbose:
                    print('last date for training additional df data', additional_ts_stacked_series.time_index[-1])
                all_ts_stacked_series = additional_ts_stacked_series
            else:
                return "Error. More than one time series for _add_additional_training_dfs not implemented"
        return additional_ts_transformers, all_ts_stacked_series.stack(ts_stacked_series)
    
    def _check_ml_prediction(self, end_time,   tcn_first_epochs=600, tcn_sub_epochs=100, nbeats_epochs=2,
                             verbose = False) -> float:
        """train ML model to predict price movement over the last self.ml_lookback_windows days 
            predicting over the next self.ml_prediction_n_days days in teh future
        end_time: the current date to predict up to
        output_chunk_length: number of predicts to make
        tcn_first_epochs: n of training epochs for the first training run
        tcn_sub_epochs: n of training epochs for subsequents training runs
        nbeats_epochs: epochs for NBEATS model
        """        
        if not self.have_trained_ml_models: # create models once, retrain incrementally
            for lookback_window in self.ml_lookback_windows:
                
                for model_n in self.model_name:
                    if model_n == "NBEATS":
                        self.ml_models_dict[str(lookback_window)+"_NBEATS"] = NBEATSModel(input_chunk_length=lookback_window, 
                                                                      output_chunk_length=self.ml_prediction_n_days,
                                                                       random_state=0,
                                                                       model_name = str(lookback_window) + "_nbeats",
                                                                       num_blocks=3,
                                                                       layer_widths=512,
                                                                      force_reset=True)
                    elif model_n == "TCN":
                        self.ml_models_dict[str(lookback_window)+"_TCN"] = TCNModel(
                            dropout=.1,
                            random_state=0,
                                dilation_base=2, 
                            weight_norm=True,
                            kernel_size=2,
                            num_filters=3,
                            num_layers=5,
                            input_chunk_length=lookback_window,
                            output_chunk_length=self.ml_prediction_n_days,
                        model_name = str(lookback_window) + "_tcn",
                        force_reset=True)
                    else:
                        raise ValueError 
                        print(f"Error. Incorrect input model of {self.model_name}")       

        training_df = self.df[self.df.index <= pd.to_datetime(end_time)]
        # add in any additional DFs, like ETHER
        additional_dfs_sliced = []
        if len(self.additional_dfs) > 0:
            for additional_df in self.additional_dfs:
                additional_dfs_sliced.append(additional_df[additional_df.index <= pd.to_datetime(end_time)])
        
        # combine TS from both DFs
        ts_transformers, ts_stacked_series, train_close_series = self._scale_time_series_df_and_time_cols(training_df)
        if verbose:
            print('original DF training series',         ts_stacked_series.components)
            print('last date for training data', ts_stacked_series.time_index[-1])
        

        if len(self.additional_dfs) >0 :
            # overwrite the ts_stacked_series var if we have additional DFS
            additional_ts_transformers, ts_stacked_series =  self._add_additional_training_dfs(ts_stacked_series, additional_dfs_sliced)
            # TODO: in the future, combine the ts_tra
            ts_transformers = {**additional_ts_transformers, **ts_transformers} # merge dicts
        
        if verbose:
            print('all series now stacked', ts_stacked_series.components)

        # train the model & make predictions
        all_predictions = []
        for lookback_name, model in self.ml_models_dict.items():
            if self.first_run and lookback_name == str(lookback_window)+"_TCN":
                model.fit(series = train_close_series, past_covariates = [ts_stacked_series], 
                          verbose=verbose, epochs=tcn_first_epochs)
            elif lookback_name == str(lookback_window)+"_NBEATS":
                model.fit(series = train_close_series, past_covariates = [ts_stacked_series], 
                          verbose=verbose, epochs=nbeats_epochs)                
            elif lookback_name == str(lookback_window)+"_TCN":
                model.fit(series = train_close_series, past_covariates = [ts_stacked_series], 
                          verbose=verbose, epochs=tcn_sub_epochs)
            
            ml_prediction = model.predict(n=self.ml_prediction_n_days, 
                    series=train_close_series, past_covariates = [ts_stacked_series]).last_value() # grab the last value
            print(f" Lookback = {lookback_name} Prediction = {ml_prediction}")
            all_predictions.append(ml_prediction)
        self.first_run = False

        
        return np.mean(all_predictions) # average the predictions
        

    def _calculate_positions(self):
        self.df['Position'] = None
        self.df['Mode'] = None
        self.df['ML_Future_Prediction']=None
        self.number_of_trades = 0
        buy_has_crossed_mean = False
        short_has_crossed_mean = False
        self.ml_prediction_date_and_price = {}
        self.mode = 'no_position' # the curent position we have
        
        
        
        for index in range(len(self.df)):
            
            row = self.df.iloc[index]
            prev_row = self.df.iloc[index - 1]
            
            
            if pd.to_datetime(row.name) < self.prediction_start: # lookback windows needed for ML model
                self.df.iloc[index, self.df.columns.get_loc('Mode')] =  self.mode
                continue 
                    
            if index == 0:
                self.df.iloc[index, self.df.columns.get_loc('Mode')] =  self.mode
                continue
            
            # update stop loss
            if self.mode == 'buy' and (1-self.stop_loss_pct)*row['close'] > self.stop_loss_price:
                self.stop_loss_price = (1-self.stop_loss_pct)*row['close']
                print(f"Updating stop loss to {self.stop_loss_price}")
                print(row['close'], 'row close')
            
            if self.mode == 'short' and (1+self.stop_loss_pct)*row['close'] < self.stop_loss_price:
                self.stop_loss_price = (1+self.stop_loss_pct)*row['close']
                print(f"Updating stop loss to {self.stop_loss_price}")
                print(row['close'], 'row close')

                                                           
            
            # check if we've previously crossed the mean trailing price
            if self.mode == 'buy'  and row['close'] > row['Rolling Mean']:
                buy_has_crossed_mean = True
                
            if self.mode == 'short'  and row['close'] < row['Rolling Mean']:
                short_has_crossed_mean = True
            

                
            # stop loss, get out of buy position
            if self.mode == 'buy' and self.stop_loss_price > row['close']:
                print('----')
                print('stop loss activated for getting out of our buy')
                print(row.name, 'current date')
                print(row['close'], 'row close')
                print(self.stop_loss_price, 'self.stop_loss_price')
                print(self.buy_entry_price, 'self.buy_entry_price')

                self.df.iloc[index, self.df.columns.get_loc('Position')] = 1
                if index + 1  == len(self.df):
                    self.df.iloc[index, self.df.columns.get_loc('Position')] = 0 
                else:
                    self.df.iloc[index + 1, self.df.columns.get_loc('Position')] = 0 # for pct change it does a ffilll. ffill with zeros

                self.df.iloc[index, self.df.columns.get_loc('Mode')] = 'buy_to_no_position'
                self.mode = 'no_position'
                buy_has_crossed_mean = False

            # stop loss, get out of short position
            elif self.mode == 'short' and self.stop_loss_price  <row['close'] :
                print('----')
                print('stop loss activated for getting out of our short')
                print(row.name, 'current date')
                print(row['close'], 'row close')
                print(self.stop_loss_price, 'self.stop_loss_price')
                print(self.short_entry_price, 'self.short_entry_price')

                self.df.iloc[index, self.df.columns.get_loc('Position')] = -1
                if index + 1  == len(self.df):
                    self.df.iloc[index, self.df.columns.get_loc('Position')] = 0 
                else:
                    self.df.iloc[index + 1, self.df.columns.get_loc('Position')] = 0 # for pct change it does a ffilll. ffill with zeros

                self.df.iloc[index, self.df.columns.get_loc('Mode')] = 'short_to_no_position'
                self.mode = 'no_position'
                short_has_crossed_mean = False



            # open -> no_position? no position is below running mean
            # or, if we are above the top band (mean reversion)
            elif self.mode == 'buy'  and (
                (row['close'] < row['Rolling Mean'] and buy_has_crossed_mean) or 
                (row['close'] > row['Bollinger High']) or 
                (row['close'] < row['Bollinger Low']) or 
                (row['Rolling Mean'] < self.buy_entry_price)):
                print('---------')
                print('checking if we should get out of our buy position')
                print(row.name, 'current date')
                print(row['Rolling Mean'], 'mean')
                print(self.buy_entry_price, 'self.buy_entry_price')
                print(row['close'], 'current close')

                
                # check ML predicted trend as well
                try:
                    ml_pred = self._check_ml_prediction(row.name)
                except ValueError: # don't have enough data for ML prediction
                    print('Ran into not enough data ValueError for buy_to_no_position')
                    continue
                print(ml_pred, 'ml_pred')
                
                if (ml_pred < row['Rolling Mean']) or \
                     (ml_pred < self.buy_entry_price) or \
                    (row['Rolling Mean'] < self.buy_entry_price):
                    print('buy_to_no_position')
                    self.df['ML_Future_Prediction'] = ml_pred

                    self.df.iloc[index, self.df.columns.get_loc('Position')] = 1

                    if index + 1  == len(self.df):
                        self.df.iloc[index, self.df.columns.get_loc('Position')] = 0 
                    else:
                        self.df.iloc[index + 1, self.df.columns.get_loc('Position')] = 0 # for pct change it does a ffilll. ffill with zeros
                    self.df.iloc[index, self.df.columns.get_loc('Mode')] = 'buy_to_no_position'

                    self.mode = 'no_position'
                    buy_has_crossed_mean = False
                else:
                    self.df.iloc[index, self.df.columns.get_loc('Mode')] = self.mode
                    
            
            # short -> no_position? no position if above running mean
            # or, if we are below the bottom band (mean reversion)
            elif self.mode == 'short'  and (
                (row['close'] > row['Rolling Mean'] and short_has_crossed_mean) or 
                (row['close'] < row['Bollinger Low']) or 
                (row['close'] > row['Bollinger High']) or
                row['Rolling Mean'] > self.short_entry_price):
                print('---------')
                print('checking if we should get out of our short position')
                print(row.name, 'current date')
                print(row['Rolling Mean'], 'mean')
                print(self.short_entry_price, 'self.short_entry_price')
                print(row['close'], 'current close')
                
                # check ML predicted trend as well
                try:
                    ml_pred = self._check_ml_prediction(row.name)
                except ValueError: # don't have enough data for ML prediction
                    print('Ran into not enough data ValueError for short_to_no_position')
                    continue
                print(ml_pred, 'ml_pred')
                if (ml_pred > row['Rolling Mean']) \
                    or (ml_pred > self.short_entry_price) \
                    or (row['Rolling Mean'] > self.short_entry_price):
                    print('short_to_no_position')
                    self.df['ML_Future_Prediction'] = ml_pred

                    self.df.iloc[index, self.df.columns.get_loc('Position')] = -1
                    if index +1  == len(self.df):
                        self.df.iloc[index, self.df.columns.get_loc('Position')] = 0 
                    else:
                        self.df.iloc[index + 1, self.df.columns.get_loc('Position')] = 0 # for pct change it does a ffilll. ffill with zeros
                    self.df.iloc[index, self.df.columns.get_loc('Mode')] = 'short_to_no_position'
                    self.mode = 'no_position'
                    short_has_crossed_mean = False
                else:
                    self.df.iloc[index, self.df.columns.get_loc('Mode')] = self.mode

            # buy check with ML model
            elif self.mode == 'no_position' and row['close'] < row['Bollinger Low'] and prev_row['close'] > prev_row['Bollinger Low']:
                self._check_if_we_should_buy(index, row)

            # short?
            elif self.mode == 'no_position' and  row['close'] > row['Bollinger High'] and prev_row['close'] < prev_row['Bollinger High']:
                self._check_if_we_should_short(index, row)
        
                       
            else:
                self.df.iloc[index, self.df.columns.get_loc('Mode')] = self.mode
                
    def _check_if_we_should_buy(self, index, row):
        """
        Determine if we should enter a buy position
        """
        print('----------')
        print('buy')
        print(row.name, 'current date')
        print(row['close'], 'close')
        # check ML predicted trend as well
        try:
            ml_pred = self._check_ml_prediction(row.name)
        except ValueError: # don't have enough data for ML prediction
            print('Ran into not enough data ValueError for buy')
            self.df.iloc[index, self.df.columns.get_loc('Mode')] = self.mode
            return 9999999

        print(ml_pred, 'ml prediction day')
        print(row['Rolling Mean'], 'mean')

        if ml_pred > row['Rolling Mean']:
            print(f"ml pred higher than mean taking position")
            self.df['ML_Future_Prediction'] = ml_pred
            self.ml_prediction_date_and_price[row.name + timedelta(days=self.ml_prediction_n_days)] = ml_pred 

            self.df.iloc[index, self.df.columns.get_loc('Position')] = 0 # buy. add one to index so that pct_change works

            if index +1  == len(self.df):
                self.df.iloc[index, self.df.columns.get_loc('Position')] = 1 
            else:
                self.df.iloc[index + 1, self.df.columns.get_loc('Position')] = 1 # buy. add one to index so that pct_change works
            self.df.iloc[index, self.df.columns.get_loc('Mode')] = 'buy'
            self.number_of_trades  +=1
            self.mode = 'buy'    
            self.buy_entry_price = row['close']
            self.stop_loss_price = row['close']*(1-self.stop_loss_pct)
        else:
            self.df.iloc[index, self.df.columns.get_loc('Mode')] = self.mode
            

            
            
    def _check_if_we_should_short(self, index, row):
        """
        Check if we should enter a short position
        """
        print('----------')
        print('short')
        print(row.name, 'current date')
        print(row['close'], 'close')
        # check ML predicted trend as well
        try:
            ml_pred = self._check_ml_prediction(row.name)
        except Exception as e: # don't have enough data for ML prediction
            print('Ran into not enough data ValueError for short')
            print(e)
            self.df.iloc[index, self.df.columns.get_loc('Mode')] = self.mode
            return 0

        print(ml_pred, 'ml pred day')
        print(row['Rolling Mean'], 'mean')

        if ml_pred < row['Rolling Mean']:
            print('pred 7 day lower than mean taking position')
            self.df['ML_Future_Prediction'] = ml_pred
            self.ml_prediction_date_and_price[row.name + timedelta(days=self.ml_prediction_n_days)] = ml_pred 

            self.df.iloc[index, self.df.columns.get_loc('Position')] = 0 #  short starts at the end of the day. Calculate pct_change starting tomorrow
            self.df.iloc[index + 1, self.df.columns.get_loc('Position')] = -1 #  short starts at the end of the day. Calculate pct_change starting tomorrow
            if index +1  == len(self.df):
                self.df.iloc[index, self.df.columns.get_loc('Position')] = -1
            else:
                self.df.iloc[index + 1, self.df.columns.get_loc('Position')] = -1 # buy. add one to index so that pct_change works
            self.df.iloc[index, self.df.columns.get_loc('Mode')] = 'short'
            self.number_of_trades  += 1
            self.mode = 'short' 
            self.short_entry_price = row['close']
            self.stop_loss_price = row['close']*(1+self.stop_loss_pct)
        else:
            self.df.iloc[index, self.df.columns.get_loc('Mode')] = self.mode
        
                
    def _build_bollinger_bands(self):
        rolling_mean = self.df['close'].rolling(self.window).mean()
        rolling_std = self.df['close'].rolling(self.window).std()

        self.df['Rolling Mean'] = rolling_mean
        self.df['Bollinger High'] = rolling_mean + (rolling_std * self.no_of_std)
        self.df['Bollinger Low'] = rolling_mean - (rolling_std * self.no_of_std)
        
        new_additional_dfs = []
        if len(self.additional_dfs) >0: 
            for df in self.additional_dfs:
                rolling_mean = df['close'].rolling(self.window).mean()
                rolling_std = df['close'].rolling(self.window).std()

                df['Rolling Mean'] = rolling_mean
                df['Bollinger High'] = rolling_mean + (rolling_std * self.no_of_std)
                df['Bollinger Low'] = rolling_mean - (rolling_std * self.no_of_std)
                
                new_additional_dfs.append(df)
        self.additional_dfs = new_additional_dfs
        
    def _calculate_returns(self):
        
        self.df['Original Position'] = self.df['Position']
        self.df['Position'].fillna(method='ffill', inplace=True)
        self.df['Market Return'] = self.df['close'].pct_change()
        self.df['Strategy Return'] = self.df['Market Return'] * self.df['Position']

    def _plot_returns(self):
        self.df['Strategy Return'].cumsum().plot(figsize=self.figsize)
    
    def _slice_df(self):
        self.df = self.df.loc[pd.to_datetime(self.from_date):, :].copy()
        self.prediction_start = pd.to_datetime(self.from_date) + timedelta(self.max_looback)
        
        sliced_additional_dfs = []
        if len(self.additional_dfs) > 0:
            for add_df in self.additional_dfs:
                sliced_df = add_df.loc[pd.to_datetime(self.from_date):, :].copy()
                sliced_additional_dfs.append(sliced_df)

        self.additional_dfs = sliced_additional_dfs
        
        

    def simulate(self):
        self._slice_df()
        self._build_bollinger_bands()
        self._calculate_positions()
        self._calculate_returns()
        self._plot_returns()
        self.end_time = time.time()
        
        print(f" Minutes taken = {(self.end_time - self.start_time)/60}")

        return (
            self.period, 
            self.window, 
            self.no_of_std, 
            self.df['Strategy Return'].sum(),
            self.number_of_trades
        )

In [ ]:

simulator = BollingerBandsSimulator(
    bitcoin_df, 
    from_date="2019-1-01", 
    period="24H", 
    window=10, 
    no_of_std=1,
    ml_lookback_windows=[31],
    ml_prediction_n_days=30,
    additional_dfs = [etherum_df],
    stop_loss_pct=.10,
    model_name=["TCN", "NBEATS"]
)
simulator.simulate()

----------
buy
2019-02-06 00:00:00 current date
3413.77 close
Ran into not enough data ValueError for buy
----------
short
2019-02-08 00:00:00 current date
3666.78 close
Ran into not enough data ValueError for short
__len__() should return >= 0
----------
short
2019-02-18 00:00:00 current date
3915.71 close
Ran into not enough data ValueError for short
__len__() should return >= 0
----------
buy
2019-03-04 00:00:00 current date
3761.56 close
 Lookback = 31_TCN Prediction = 3983.130904088883
 Lookback = 31_NBEATS Prediction = 8054.145784648234
6018.638344368559 ml prediction day
3872.8599999999997 mean
ml pred higher than mean taking position
Updating stop loss to 3506.742
3896.38 row close
---------
checking if we should get out of our buy position
2019-03-05 00:00:00 current date
3848.245 mean
3761.56 self.buy_entry_price
3896.38 current close
 Lookback = 31_TCN Prediction = 4567.122061532147
 Lookback = 31_NBEATS Prediction = 8133.427020356921
6350.274540944534 ml_pred
Updating stop 

In [ ]:
simulator

#### Visualize Trades

In [ ]:
DATE_FILTER = '2017-10-22'
simulator.df[simulator.df.index > DATE_FILTER ]  [['Rolling Mean', 'Bollinger High', 'Bollinger Low', 'close']].plot(figsize=(19,8))

prev_pos = None
for index, pos in simulator.df[simulator.df.index > DATE_FILTER ]['Mode'].iteritems():
    if pos == "short":
        plt.axvline(index, color='red', linewidth=.3)
    if pos == "buy":
        plt.axvline(index, color='green', linewidth=.3)
    if pos == 'short_to_no_position':
        plt.axvline(index, color='black', linewidth=3)
    if pos == 'buy_to_no_position':
        plt.axvline(index, color='black', linewidth=3, label = 'buy_to_no_postiion')


for key,val in simulator.ml_prediction_date_and_price.items():
    plt.scatter(key, val, marker="*", linewidths=4, color='brown', label='predictions')

# plt.legend()

In [48]:
# short
# 2019-11-04 00:00:00 current date
#  Lookback = 30 Prediction = 7337.89262836642
#  Lookback = 90 Prediction = 9278.555809841882
#  Lookback = 180 Prediction = 11221.047974868856
# 9279.165471025719 ml prediction 7 day
# 8554.994666666667 mean

In [48]:
simulator.df[simulator.df.index > '2021-07-22' ] 

,open,high,low,close,volume,Rolling Mean,Bollinger High,Bollinger Low,Position,Mode,ML_Future_Prediction,Original Position,Market Return,Strategy Return
date,,,,,,,,,,,,,,
2021-07-23,32305.96,33581.55,32057.89,33581.55,2.255205e+10,32257.401429,34040.339110,30474.463747,0.0,no_position,43068.207252,None,0.039255,0.000000
2021-07-24,33593.73,34490.39,33424.86,34292.45,2.166471e+10,32312.539286,34213.103963,30411.974609,0.0,no_position,43068.207252,None,0.021169,0.000000
2021-07-25,34290.29,35364.93,33881.84,35350.19,2.085669e+10,32391.825000,34525.091247,30258.558753,0.0,no_position,43068.207252,None,0.030845,0.000000
2021-07-26,35384.03,40499.68,35287.31,37337.53,5.102213e+10,32690.516429,35600.340570,29780.692287,0.0,no_position,43068.207252,None,0.056219,0.000000
2021-07-27,37276.04,39406.94,36441.73,39406.94,3.509737e+10,33169.438571,37134.137920,29204.739223,0.0,no_position,43068.207252,None,0.055424,0.000000
2021-07-28,39503.19,40816.07,38862.44,39995.91,3.870240e+10,33681.835714,38490.922663,28872.748766,0.0,no_position,43068.207252,None,0.014946,0.000000
2021-07-29,39995.45,40593.07,39352.06,40008.42,2.716715e+10,34269.527857,39616.715385,28922.340329,0.0,no_position,43068.207252,None,0.000313,0.000000
2021-07-30,40027.48,42235.55,38397.35,42235.55,3.307278e+10,35041.957143,41102.146108,28981.768178,0.0,no_position,43068.207252,None,0.055667,0.000000
2021-07-31,42196.30,42231.45,41110.83,41626.20,2.580285e+10,35762.895000,42153.418791,29372.371209,0.0,no_position,43068.207252,None,-0.014427,-0.000000


In [21]:
simulator.df[simulator.df.index > '2021-08-31' ].Mode.value_counts()

no_position    3
Name: Mode, dtype: int64

In [22]:
np.sum(simulator.df.Mode == None)

0